In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 

/tmp/ipykernel_5443/776920745.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
import tensorflow as tf 
tf.__version__

2024-02-07 16:20:16.487761: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 16:20:16.775397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 16:20:16.775698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 16:20:16.828941: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 16:20:16.940240: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 16:20:16.945386: I tensorflow/core/platform/cpu_feature_guard.cc:1

'2.15.0'

In [5]:
x = data.iloc[:,3:-1].values
y = data.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2]=le.fit_transform(x[:,2])

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))

In [27]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2)

In [28]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)

In [29]:
ann = tf.keras.models.Sequential()

In [30]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [31]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

In [32]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [33]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [34]:
ann.fit(xtrain,ytrain,batch_size=32,epochs=100)

Epoch 1/100


250/250 [==============================] - 1s 976us/step - loss: 0.5500 - accuracy: 0.7928
Epoch 2/100
250/250 [==============================] - 0s 909us/step - loss: 0.4687 - accuracy: 0.7954
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.7954
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4346 - accuracy: 0.7954
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4273 - accuracy: 0.7954
Epoch 6/100
250/250 [==============================] - 0s 854us/step - loss: 0.4208 - accuracy: 0.7972
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4137 - accuracy: 0.8089
Epoch 8/100
250/250 [==============================] - 0s 922us/step - loss: 0.4061 - accuracy: 0.8229
Epoch 9/100
250/250 [==============================] - 0s 1000us/step - loss: 0.3976 - accuracy: 0.8322
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3883 - accurac

In [35]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>.5)

1/1 [==============================] - 0s 69ms/step
[[False]]


In [36]:
res = ann.predict(xtest)
res =res >.5
print(np.concatenate((res.reshape(len(res),1),ytest.reshape(len(ytest),1)),1))


63/63 [==============================] - 0s 878us/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 1]
 [0 0]]


In [37]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(confusion_matrix(ytest,res))

[[1520   80]
 [ 202  198]]


In [38]:
accuracy_score(ytest,res)

0.859